In [87]:
# Dependencies
# !pip install langchain
# !pip install openai

In [88]:
# Standard Imports
import openai
import os
from langchain.llms import OpenAI
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.embeddings import OpenAIEmbeddings


import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

load_dotenv()
# API configuration
openai.api_key = os.getenv("OPENAI_API_KEY")

In [89]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="gpt-3.5-turbo",temperature=0.9)

# The LLM takes a prompt as an input and outputs a completion
prompt = "What is the per capita income of a indiviual in India"
completion = llm(prompt)
completion

/Users/shabbirgovernor/Library/Python/3.9/lib/python/site-packages/langchain/llms/openai.py:173: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/shabbirgovernor/Library/Python/3.9/lib/python/site-packages/langchain/llms/openai.py:753: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


'As of 2020, the per capita income in India is estimated to be around $2,104.'

In [91]:
# Set System behaviour for Chat
chat = ChatOpenAI(temperature=0)

chat([SystemMessage(content="You are a AI bot which replys user what to eat in three short sentence"),
        HumanMessage(content="Are pizzas healthy, Should I eat them?")
    ]
)

AIMessage(content="Pizzas can be enjoyed in moderation as part of a balanced diet. Opt for whole wheat crust, lean protein toppings, and plenty of vegetables to make it healthier. However, it's important to consider portion sizes and limit consumption of high-fat and processed toppings.", additional_kwargs={}, example=False)

In [94]:
# Embedding text with Langchain 
embeddings = OpenAIEmbeddings()

text = "I dont like studying. Dont make me study"
text1 = "Maria has a parrot. Where do parrots live?"
text_embedding = np.array(embeddings.embed_query(text))
text_embedding1 = np.array(embeddings.embed_query(text1))
print(text_embedding)
print(text_embedding1)

[-0.04051665  0.01633171 -0.00271362 ...  0.00470819  0.00965397
 -0.0272362 ]
[-0.00831975 -0.015002   -0.01128614 ... -0.00722388 -0.03559668
 -0.0221566 ]


In [95]:
list1 = text_embedding.reshape(1, -1)
list2 = text_embedding1.reshape(1, -1)

similarity = cosine_similarity(list1, list2)

print(similarity)

[[0.70411446]]


In [96]:
# Prompt templates in Langchain
llm("Can Narendra Modi have a conversation with Mahatma Gandhi?")
prompt = """Question: Can Narendra Modi have a conversation with Mahatma Gandhi?

Let's think step by step.

Answer: """
llm(prompt)

'Technically, Narendra Modi, being the current Prime Minister of India, cannot have a physical conversation with Mahatma Gandhi, as Gandhi passed away in 1948. However, if we are considering a hypothetical scenario in which time travel or some form of communication beyond physical existence is possible, then it would be conceivable for Narendra Modi to have a conversation with Mahatma Gandhi.'

In [104]:
# Chaining 
from langchain import PromptTemplate, OpenAI, LLMChain

prompt_template = "What is a good name for a company that makes {product}?"

llm = OpenAI(temperature=0)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
llm_chain("colorful socks")

{'product': 'colorful socks', 'text': '\n\nSocktastic!'}

In [105]:
input_list = [
    {"product": "socks"},
    {"product": "computer"},
    {"product": "shoes"}
]

llm_chain.apply(input_list)

[{'text': '\n\nCozy Toes Socks.'},
 {'text': '\n\nTechCore Solutions.'},
 {'text': '\n\nFootwear Factory.'}]

In [106]:
llm_chain.generate(input_list)

LLMResult(generations=[[Generation(text='\n\nCozy Toes Socks.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nTechCore Solutions.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nFootwear Factory.', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'total_tokens': 57, 'prompt_tokens': 36, 'completion_tokens': 21}, 'model_name': 'text-davinci-003'}, run=[RunInfo(run_id=UUID('8534a09c-e08c-4fcb-8575-3dab8f828837')), RunInfo(run_id=UUID('80f8fb9a-8296-4be6-81ad-23dde5d35e0f')), RunInfo(run_id=UUID('e7c1e9d5-120a-4f32-9cd1-5691b45e13b3'))])

In [107]:
# Single input example
llm_chain.predict(product="colorful socks")

'\n\nSocktastic!'

In [109]:
# Multiple inputs example

template = """Tell me a {adjective} joke about {subject}."""
prompt = PromptTemplate(template=template, input_variables=["adjective", "subject"])
llm_chain = LLMChain(prompt=prompt, llm=OpenAI(temperature=0))

llm_chain.predict(adjective="happy", subject="politics")

'\n\nQ: What did the politician say when he was offered a free trip to the moon?\nA: "That\'s one small step for me, one giant leap for mankind!"'

In [110]:
# Agents and Tools
"""
* When employed effectively, agents can yield significant power by utilizing an LLM to make decisions, execute actions, 
observe outcomes, and repeat this process until the desired objective is achieved.

To successfully deploy agents, it is essential to comprehend the following concepts:

Tool: A specialized function that carries out a specific task. Examples include Google Search, database lookup, Python REPL, and other tool chains.

LLM: The language model that empowers the agent and provides it with the ability to understand and generate human-like text.

Agent: The designated agent to be utilized for the task at hand.

"""


'\n* When employed effectively, agents can yield significant power by utilizing an LLM to make decisions, execute actions, \nobserve outcomes, and repeat this process until the desired objective is achieved.\n\nTo successfully deploy agents, it is essential to comprehend the following concepts:\n\nTool: A specialized function that carries out a specific task. Examples include Google Search, database lookup, Python REPL, and other tool chains.\n\nLLM: The language model that empowers the agent and provides it with the ability to understand and generate human-like text.\n\nAgent: The designated agent to be utilized for the task at hand.\n\n'

In [111]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [112]:
#!pip install wikipedia

In [113]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0)
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

In [114]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [115]:
agent.run("In what year India achieved Indepence? What is this year raised to the power 3?")



> Entering new  chain...
 I need to find out the year India achieved independence and then use a calculator to calculate the power.
Action: Wikipedia
Action Input: India Independence
Observation: Page: Indian independence movement
Summary: The Indian independence movement was a series of historic events with the ultimate aim of ending British rule in India also known as British Raj. It lasted from 1857 to 1947.
The first nationalistic revolutionary movement for Indian independence emerged from Bengal. It later took root in the newly formed Indian National Congress with prominent moderate leaders seeking the right to appear for Indian Civil Service examinations in British India, as well as more economic rights for natives. The first half of the 20th century saw a more radical approach towards self-rule by the Lal Bal Pal triumvirate, Aurobindo Ghosh and V. O. Chidambaram Pillai.
The stages independence struggle in the 1920s were characterized by the leadership of Mahatma Gandhi and Co

'India achieved independence in 1947 and this year raised to the power 3 is 7380705123.'

In [ ]:
# Memory
"""
State refers to the concept of retaining information between successive calls of a chain or agent. 
In LangChain, a standardized interface for managing state, known as memory, is provided. This memory functionality encompasses
 various implementations and includes examples of chains and agents that leverage this feature effectively.

By employing memory, chains and agents can store and access relevant information throughout their execution, 
enabling them to maintain context and make informed decisions based on past observations or actions.

The term "memory" in this context denotes the ability of the system to retain and recall data as needed, 
facilitating continuity and coherence in the chain or agent's behavior.

LangChain offers a range of memory implementations, allowing developers to choose the most suitable option for their specific 
use case. This flexibility empowers chains and agents to effectively utilize and manage state information, 
enhancing their overall performance and adaptability.
"""

In [116]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.predict(input="Hi there!")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

In [117]:
conversation.predict(input="Can we talk about climate?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: Can we talk about climate?
AI:

> Finished chain.


' Sure! What would you like to know about climate?'

In [118]:
conversation.predict(input="Is the current world climate good?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: Can we talk about climate?
AI:  Sure! What would you like to know about climate?
Human: Is the current world climate good?
AI:

> Finished chain.


" That's a difficult question to answer. The current climate is changing rapidly, and it's hard to say whether it's good or bad. However, it's clear that the climate is changing in ways that are having a negative impact on the environment and human life."

In [119]:
conversation.predict(input="Thats very sad to here")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: Can we talk about climate?
AI:  Sure! What would you like to know about climate?
Human: Is the current world climate good?
AI:  That's a difficult question to answer. The current climate is changing rapidly, and it's hard to say whether it's good or bad. However, it's clear that the climate is changing in ways that are having a negative impact on the environment and human life.
Human: Thats very sad to here
AI:

> Finished chain.


' Yes, it is. We need to take action to reduce the effects of climate change and protect our planet.'